In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
#import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
import pickle
from sklearn.model_selection import train_test_split
import torch.optim as optim
from helper_functions import train_or_load

In [39]:
from sklearn.neighbors import KNeighborsClassifier

In [2]:
#load the shadow model trained in the other python script
device = device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet_shadow =  models.mobilenet_v2(weights=None,num_classes = 200)

resnet_cifar = torch.load("shadow_models/mobilenet_shadow_tinyimage_overtrained.pth",map_location=device)
resnet_shadow.load_state_dict(resnet_cifar)

<All keys matched successfully>

In [3]:
DATA_PATH = 'pickle/tinyimagenet/mobilenetv2/shadow.p'
# Change the DATA_PATH to your local pickle file path


with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)


#splitting
#only use train set here
train_data, val_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
dataloader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=False, num_workers=2)
testloader =  torch.utils.data.DataLoader(val_data, batch_size=1, shuffle=True, num_workers=2)

for batch_idx, (img, label) in enumerate(dataloader):
    img = img.to(device)


In [4]:
#generate dataset for attack model training
resnet_shadow.eval()
dataset_attack = []
with torch.no_grad():
    for images, labels in testloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_shadow(images)
        
        #take the 3 biggest logist
        top_values = torch.topk(logits, k=3).values
        top_values, indices = torch.sort(top_values, dim=1, descending=True)
        dataset_attack.append([top_values,0])
        
with torch.no_grad():
    for images, labels in dataloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_shadow(images)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values
        top_values, indices = torch.sort(top_values, dim=1, descending=True)

        dataset_attack.append([top_values,1])
        



In [5]:

# # Convert all tensors to the same dtype first
tensors = [data[0].float() for data in dataset_attack]  # Ensure all tensors are Float type
all_data = torch.cat(tensors, dim=0)  # Concatenate all tensors

# # Calculate mean and std
mean = all_data.mean(dim=0)
std = all_data.std(dim=0)

# # Standardize data in the list
standardized_data_list = [( (data[0] - mean) / std, data[1] ) for data in dataset_attack]


In [75]:
knn_list = [

]
knn_list_labels = []
for i in range(len(standardized_data_list)):
    knn_list.append(standardized_data_list[i][0].numpy()[0])
    knn_list_labels.append(standardized_data_list[i][1])
knn_list_labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [60]:
dataloader_attack = torch.utils.data.DataLoader(standardized_data_list, batch_size=64, shuffle=False, num_workers=2) #shuffled training data

In [61]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.fc2 = nn.Linear(32, 1)
       

    def forward(self, x):
        
        x = torch.sigmoid(self.fc2(self.fc1(x)))
        return x

    

In [82]:
neigh = KNeighborsClassifier(n_neighbors=2)
neigh.fit(knn_list, knn_list_labels)

KNeighborsClassifier(n_neighbors=2)

In [85]:
pickle.dump(neigh, open("knn.pkl", "wb"))

In [25]:
attack_model = SimpleNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=0.0001)

In [26]:
#dict =  torch.load('attack_model.pth_3', map_location='cpu')
#model.load_state_dict(dict)

In [29]:
train_or_load(attack_model,dataloader_attack,optimizer,criterion,5)

Epoch: 1
Loss: 0.588577151298523
Epoch: 2
Loss: 0.5885018110275269
Epoch: 3
Loss: 0.5884440541267395
Epoch: 4
Loss: 0.5883986353874207
Epoch: 5
Loss: 0.5883615016937256


In [11]:
torch.save(attack_model.state_dict(), 'attack_models/attack_mobilenet_tinyimage.pth')

<h1>Evaluation</h1>

In [33]:
DATA_PATH = "pickle/tinyimagenet/mobilenetv2/eval.p"

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
# Convert all tensors to the same dtype first

eval_dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=1 , shuffle=False, num_workers=1)
# #splitting
# for batch_idx, (img, label, membership) in enumerate(dataloader):
#     img = img.to(device)


In [34]:
# Load target model
resnet_target_model =  models.mobilenet_v2(weights=None,num_classes = 200)
check = torch.load("models/mobilenetv2_tinyimagenet.pth", map_location=device)
resnet_target_model.load_state_dict(check["net"])
resnet_target_model.eval()

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [35]:
#get posteriors from target
target_dataset_eval = []
with torch.no_grad():
    for images,_, member in eval_dataloader: #need only one
            # Move images and labels to the appropriate device
        images, labels = images.to(device), labels.to(device)
            
            # Forward pass
        logits = resnet_target_model(images)
        
        #take the 3 biggest logist
        
        top_values = torch.topk(logits, k=3).values #order poseri
        sorted_tensor, indices = torch.sort(top_values, dim=1,descending=True)
        target_dataset_eval.append([sorted_tensor, member.item()])
        


In [36]:
# #onvert all tensors to the same dtype first
tensors = [data[0].float() for data in target_dataset_eval]  # Ensure all tensors are Float type
all_data = torch.cat(tensors, dim=0)  # Concatenate all tensors

# # Calculate mean and std
mean = all_data.mean(dim=0)
std = all_data.std(dim=0)

# # Standardize data in the list
dataset = [( (data[0] - mean) / std, data[1] ) for data in target_dataset_eval]

In [83]:
knn_list = [

]
knn_list_labels = []
for i in range(len(dataset)):
    knn_list.append(dataset[i][0].numpy()[0])
    knn_list_labels.append(dataset[i][1])
knn_list_labels

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [84]:
c = 0
for i,member in zip(knn_list,knn_list_labels):
    sol = neigh.predict([i])
    print(sol)
    if sol == member:

        c +=1

c/len(knn_list)

[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[0]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]


0.995

In [37]:
dataloader_eval = torch.utils.data.DataLoader(dataset, batch_size=1 , shuffle=False, num_workers=2)

In [38]:
def evaluate(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for data in dataloader:
            inputs = data[0]
            
            labels = data[1].float()
            labels = labels.float()
            optimizer.zero_grad()

            outputs = model(inputs.squeeze(dim=1))
            predicted = torch.round(outputs)  # Round the outputs to 0 or 1
            print(predicted,labels)

            total += labels.size(0)  # Increment the total count by batch size
            correct += (predicted == labels).sum().item()  # Count correct predictions

    accuracy = correct / total
    return accuracy
accuracy = evaluate(attack_model, dataloader_eval)

print(f'Accuracy: {accuracy:.2f}')

tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tensor([1.])
tensor([[1.]]) tenso